In [1]:
import os
os.chdir("..")

In [2]:
import django
# In case that we need it later
# from django.conf import settings
os.environ.setdefault('DJANGO_SETTINGS_MODULE',
'mimuse.settings')
# This is for async, in case we will see it later (maybe)
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()


In [3]:
import json
import http.client
from songs.models import Song, Album
from artists.models import Artist
from datetime import datetime

In [4]:
import http.client

conn = http.client.HTTPSConnection("spotify23.p.rapidapi.com")

headers = {
    'X-RapidAPI-Key': "208491c007msh75bd41f34ef037fp1815aejsn15b1bb64cbfe",
    'X-RapidAPI-Host': "spotify23.p.rapidapi.com"
}

conn.request("GET", "/recommendations/?limit=5&seed_tracks=38MKW2tQHtyO8djIOKlEFF", headers=headers)

res = conn.getresponse()
data = json.loads(res.read().decode("utf-8"))

# print(data.decode("utf-8"))

In [5]:
song_list = list()

track = data['tracks']
for song in data['tracks']:
    # print(song['album']['release_date'])
    if(song['album']['release_date_precision']=="day"):
        song_list.append(Song(api_id=song['id'], title = song['name'], preview_url= song['preview_url'], release_date= song['album']['release_date'], popularity=song['popularity']))
    else:
        song_list.append(Song(api_id=song['id'], title = song['name'], preview_url= song['preview_url'], popularity=song['popularity']))
    # print (song)
# print(song_list)
bulk = Song.objects.bulk_create(song_list)

In [6]:
for song in data['tracks']:
    print(song)
    id = str(song['artists'][0]['id'])
    song_id = song['id']
    print(song_id)
    if Artist.objects.filter(api_id=id).exists():
        print('Developer msg: data already exists in database')
        continue
    else:
        conn.request("GET", "/artists/?ids={id}".format(
                        id=id), headers=headers)
        res = conn.getresponse()
        datas = json.loads(res.read().decode('utf-8'))['artists'][0]
        artist = Artist(api_id=id, name = datas['name'], popularity=datas['popularity'], image=datas['images'][0]['url'], followers=datas['followers']['total'])
        artist.save()
        song_id = song['id']
        this_song = Song.objects.get(api_id=song_id)
        print(this_song)
        artist.songs.add(this_song)
        artist.save()



{'album': {'album_type': 'SINGLE', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0XATRDCYuuGhk0oE7C0o5G'}, 'id': '0XATRDCYuuGhk0oE7C0o5G', 'name': 'MAMAMOO', 'type': 'artist', 'uri': 'spotify:artist:0XATRDCYuuGhk0oE7C0o5G'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS', 'SI', 'KR', 'BD', 'PK', 'LK', 'GH', 'KE', 'NG', 'TZ', 'UG', 'AG', 'AM', 'BS', 'BB', 'BZ', 'BT', 'BW', 'BF', 'CV', 'CW', 'DM', 'FJ', 'GM', 'GE', 'GD', 'GW', 'GY', 'HT', 'JM', 'KI', 'LS', 'LR', 'MW

In [23]:
artists = Artist.songs.through.objects.filter(song_id=571)
print(artists)
# for artist in artists:
#     print(artist.artist_id)
#     print(Artist.objects.get(id=artist.artist_id))

<QuerySet []>


In [ ]:
## get artist singles
for artist in Artist.objects.all():
    api_id = artist.api_id

    conn = http.client.HTTPSConnection("spotify23.p.rapidapi.com")

    headers = {
        'X-RapidAPI-Key': "208491c007msh75bd41f34ef037fp1815aejsn15b1bb64cbfe",
        'X-RapidAPI-Host': "spotify23.p.rapidapi.com"
    }

    conn.request("GET", "/artist_singles/?id={api_id}".format(api_id=api_id), headers=headers)
    print(api_id)
    res = conn.getresponse()
    singles = json.loads(res.read().decode('utf-8'))['data']['artist']['discography']['singles']['items']
    for single in singles: 
        if single['releases']['items'][0]['type'] == 'SINGLE':
            id = single['releases']['items'][0]['id']
            if Song.objects.filter(api_id=id).exists():
                continue
            else:
                print(id)
                try:
                    conn.request("GET", "/albums/?ids={id}".format(id=id), headers=headers)

                    res = conn.getresponse()
                    song_info = json.loads(res.read().decode("utf-8"))['albums'][0]['tracks']['items'][0]
                    print(song_info)
                    song_record = Song.objects.update_or_create(api_id=song_info['id'],title=song_info['name'], popularity=song_info['popularity'], preview_url=song_info['preview_url'] )
                    song_record.save()
                    song_artist = Artist.objects.get(api_id=api_id)
                    song_artist.songs.add(song_record)
                    song_artist.save()
                except:
                    continue
        else:
            continue


In [29]:
Song.objects.all().delete()
Artist.objects.all().delete()
Album.objects.all().delete()

(9, {'songs.Album': 9})